In [19]:

import pandas as pd
import numpy as np
import scipy.interpolate as interp
from datetime import timedelta, datetime
from icecream import ic
import os

In [189]:
# Input paths
dir = "C:/Users/asobchen/AppData/Local/Programs/Python/Python39/alberta/"
# dirData = "C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/"
dirData = "C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Nearby/filled/"


year = 2023
turbine_name = 'Enmax_Taber'
pc_dir = 'C:/Users/asobchen/AppData/Local/Programs/Python/Python39/Power Curves/full/'

# Output path
dirOut = "C:/Users/asobchen/AppData/Local/Programs/Python/Python39/alberta/outputs/"
sunrise_sunset_file = dir+"Sun.csv"

# Load station data from Nearby_updated_111.csv
stations_path = os.path.join(dir, 'Nearby_updated_2.csv')
wind_turbines = pd.read_csv(stations_path)

# Extract hub height and number of stations for the turbine
hub_height = wind_turbines.loc[wind_turbines['Asset Name'].str.contains(turbine_name, case=False), 'hub_height'].values[0]
number_of_turbines = wind_turbines.loc[wind_turbines['Asset Name'].str.contains(turbine_name, case=False), 'number_of_turbines'].values[0]
pcf_name = wind_turbines.loc[wind_turbines['Asset Name'].str.contains(turbine_name, case=False), 'Model'].values[0]
project_name = wind_turbines.loc[wind_turbines['Asset Name'].str.contains(turbine_name, case=False), 'Nearby_Station'].values[0]
capacity_MW = wind_turbines.loc[wind_turbines['Asset Name'].str.contains(turbine_name, case=False), 'total_capacity_MW'].values[0]
capacity = capacity_MW * 1000
# metdata_name = project_name + '_' + str(year) +'.csv'
metdata_name_f = project_name + '_' + str(year) +'_filled.csv'
# metdata_name_li = project_name + '_' + str(year) +'_filled_li.csv'
# Load data
power_curve = np.array(pd.read_table(os.path.join(pc_dir, pcf_name+'.txt'), header=0))
metdata = pd.read_csv(os.path.join(dirData, metdata_name_f))
# metdata_f = pd.read_csv(os.path.join(dirData_f, metdata_name_f))
# metdata_li = pd.read_csv(os.path.join(dirData_f, metdata_name_li))



# Constants
rho_std, R = 1.225, 287.05
ref_height = 10
alpha = 0.143
losses = 0
w_speed = 5.0


In [190]:
# Load sunrise and sunset data from Sun1.xlsx
sun_times = pd.read_csv(sunrise_sunset_file)
# Assuming sun_times is your DataFrame
columns = sun_times.columns.tolist()

# Convert all column names to strings
sun_times.columns = sun_times.columns.astype(str)

# Specify the format for date parsing and convert the 'Date' column to datetime
sun_times['date'] = pd.to_datetime(sun_times['date'], format='%b %d %Y', errors='coerce')

# Convert the 'Date' column to datetime, then replace the year with 2014
sun_times['date'] = pd.to_datetime(sun_times['date'])
sun_times['date'] = sun_times['date'].apply(lambda x: x.replace(year))


sun_times_turbine = sun_times[sun_times['turbine_name'] == turbine_name]



def blanket(df, start_date='07-15', end_date='10-01', year_now=year):
    start_date = str(year_now)  + '-' + start_date
    end_date =  str(year_now) + '-' + end_date

    # Step 2: Convert start_date and end_date to datetime objects
    start_date = pd.to_datetime(start_date, format='%Y-%m-%d')
    end_date = pd.to_datetime(end_date, format='%Y-%m-%d')

    # Step 3: Filter the DataFrame to include rows within the date range
    df_intime = df[(df['date'] >= start_date) & (df['date'] <= end_date)]


    df_intime['rise'] = pd.to_datetime(df_intime['date'].astype(str) + ' ' + df_intime['rise'])
    df_intime['set'] = pd.to_datetime(df_intime['date'].astype(str) + ' ' + df_intime['set'])

    # Calculate 1 hour before 'rise' and 1 hour after 'set'
    df_intime['1_hour_after_rise'] = df_intime['rise'] + timedelta(hours=1)
    df_intime['1_hour_before_set'] = df_intime['set'] - timedelta(hours=1)

    return df_intime


df_blanket = blanket(sun_times_turbine)







C:\Users\asobchen\AppData\Local\Temp\ipykernel_21444\2822573230.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_intime['rise'] = pd.to_datetime(df_intime['date'].astype(str) + ' ' + df_intime['rise'])
C:\Users\asobchen\AppData\Local\Temp\ipykernel_21444\2822573230.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_intime['set'] = pd.to_datetime(df_intime['date'].astype(str) + ' ' + df_intime['set'])
C:\Users\asobchen\AppData\Local\Temp\ipykernel_21444\2822573230.py:37: SettingWithCopyWarning: 


In [191]:
# Extract data from meteorological data
WindSp_kmh = metdata['Wind Spd (km/h)']  # Wind speed at 10 meters in km/h
WindSp = WindSp_kmh * 0.27778  # Convert to m/s
windDir = metdata['Wind Dir (10s deg)'] 
T2m = metdata['Temp (°C)'] 
Precipitation = metdata['Precip. Amount (mm)']  # Assuming precipitation data exists in metdata
 


metdata['Date/Time (LST)'] = pd.to_datetime(metdata['Date/Time (LST)'])

# Function to map months to seasons and return roughness value for a specific turbine
def get_roughness(row, wind_turbines, turbine_name):
    month = row['Date/Time (LST)'].month
    # Use turbine_name to find the corresponding wind farm roughness
    if turbine_name in wind_turbines['Asset Name'].values:
        roughness_row = wind_turbines[wind_turbines['Asset Name'] == turbine_name].iloc[0]
        
        if month in [6, 7]:  # June, July
            return roughness_row['Summer Jun-Jul']
        elif month == 8:  # August
            return roughness_row['Pre-harvest Aug']
        elif month in [9, 10, 11]:  # September, October, November
            return roughness_row['Post-harvest/pre-snow Sep-Oct']
        elif month in [12, 1, 2]:  # December, January, February
            return roughness_row['Snow covered Dec-Feb']
        elif month in [3, 4, 5]:  # March, April, May
            return roughness_row['Spring Mar-May']
    
    return None  # In case no roughness is found for the turbine

metdata['Roughness'] = metdata.apply(get_roughness, axis=1, wind_turbines=wind_turbines, turbine_name=turbine_name)

SR_ian = metdata['Roughness']


# Function to calculate wind speed at hub height using power law fsr,
def wind_speed_at_hub_height(WindSp, h_hub, fsr, h_ref=10, alpha=0.143):
    return WindSp * ((np.log(h_hub / fsr)) / (np.log(h_ref / fsr))) 
    # return WindSp * (h_hub / h_ref) ** alpha

# Function to calculate air density
def calculate_air_density(surface_pressure, T):
    if T <= -273.15:  # Temperature can't be lower than absolute zero
        T = -273.15
    return surface_pressure / (R * (T + 273.15))  # Convert T to Kelvin

# Function to calculate power output
def power_output(W_hub, rho_site, power_curve, losses):
    adj_factor = (rho_site / rho_std)**(1. / 3)
    finter = interp.interp1d(power_curve[:, 0], power_curve[:, 1], kind='linear', bounds_error=False, fill_value=0)
    power = finter(W_hub * adj_factor) * (1 - losses)
    return float(power), adj_factor

# Calculate wind speed at hub height and power output for each hour
result = []
for i in range(len(T2m)):
    W_hub = wind_speed_at_hub_height(WindSp[i], hub_height, SR_ian[i], ref_height, alpha)  # Calculate W_hub using WindSp  SR[i],
    rho_site = rho_std
    power_out, adj_factor = power_output(W_hub, rho_site, power_curve, losses)
    result.append({
        'time': metdata['Date/Time (LST)'][i],
        'temp': metdata['Temp (°C)'][i],
        'precip': metdata['Precip. Amount (mm)'][i],
        'WindSp': WindSp[i],
        'W_hub': W_hub,
        'power_out': power_out
    })

# Create DataFrame from results
df_results_ian = pd.DataFrame(result)


# # Ensure the 'time' column is of datetime type
df_results_ian['time'] = pd.to_datetime(df_results_ian['time'])

# Format the 'time' column to 'YYYY-MM-DD HH:MM:SS'
df_results_ian['time'] = df_results_ian['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
# Ensure the '1_hour_after_rise' and '1_hour_before_set' are datetime objects if they are not already
df_results_ian['time'] = pd.to_datetime(df_results_ian['time'])

df_blanket['1_hour_after_rise'] = pd.to_datetime(df_blanket['1_hour_after_rise'])
df_blanket['1_hour_before_set'] = pd.to_datetime(df_blanket['1_hour_before_set'])

# Ensure df_blanket['date'] is in date format
df_blanket['date'] = pd.to_datetime(df_blanket['date']).dt.date



def win_speed_work(row, speed):

    for i in speed:
    #     ic('---------', i)
        # Loop through filtered_results and modify the power_out column if W_hub <= 5.0
        if pd.notna(row['W_hub']) and float(row['W_hub']) <= i:
            # row['power_out'] = 0.0  # Modify the row's power_out value
            row[f'blanket_{i}'] = 0.0

        # Loop through filtered_results and modify the power_out column if W_hub <= 5.0
        if pd.notna(row['W_hub']) and float(row['W_hub']) <= i and row['temp'] > 9.5 and row['precip'] < 1:
            # row['power_out'] = 0.0  # Modify the row's power_out value
            row[f'smart_{i}'] = 0.0



    return row

# if 15-07 and 30 -09 - turbine not work - power = 0.0
def datework_row(row, start_date='07-15', end_date='10-01', year_now=year, speed=[]):
    start_date = str(year_now)  + '-' + start_date
    end_date =  str(year_now) + '-' + end_date

    # Step 2: Convert start_date and end_date to datetime objects
    start_date = pd.to_datetime(start_date, format='%Y-%m-%d')
    end_date = pd.to_datetime(end_date, format='%Y-%m-%d')

    if end_date >= row['time'] >=  start_date:
        # Check if the date part of row['time'] exists in df_blanket['date']
        if row['time'].date() in df_blanket['date'].values:
            # Print the row if both conditions are satisfied
            blanket_rows_e = df_blanket[df_blanket['date'] == row['time'].date()]
            # blanket_rows_m = df_blanket[df_blanket['date'] == row['time'].date() + timedelta(days=1)]

            [b_rise] = blanket_rows_e['1_hour_after_rise'].values
            [a_set] = blanket_rows_e['1_hour_before_set'].values

            if b_rise >= row['time']:
                # ic(row['time'])
                row = win_speed_work(row, speed)

            if row['time'] >= a_set:
                # ic(row['time'])
                row = win_speed_work(row, speed)

    return row


# temp < 9.5

# Precip. Amount (mm) > 1


win_speeds = [5, 5.5, 6, 6.5, 7, 7.5, 8]
df_results_ian
# for i in win_speeds:
#     ic('---------', i)


speed_res = []

for speed in win_speeds:
        df_results_ian[f'blanket_{speed}'] = df_results_ian['power_out']
        df_results_ian[f'smart_{speed}'] = df_results_ian['power_out']



for idx, row in df_results_ian.iterrows():
    res_date = datework_row(row, '07-15', '10-01',  year_now=year, speed=win_speeds) 
    speed_res.append(res_date)

speed_res_df = pd.DataFrame(speed_res)
# speed_res_df['time'] = pd.to_datetime(speed_res_df['time'])

speed_res_df.to_csv(dirOut + turbine_name +'_' + str(year) + '_power_output_new.csv', index=False)









ic('---------')



# w_select = filtered_results[filtered_results['W_hub'] <= w_speed]






# df_blanket.to_csv('Magrath_2014.csv', index=False)


C:\Users\asobchen\AppData\Local\Temp\ipykernel_21444\619061979.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_blanket['1_hour_after_rise'] = pd.to_datetime(df_blanket['1_hour_after_rise'])
C:\Users\asobchen\AppData\Local\Temp\ipykernel_21444\619061979.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_blanket['1_hour_before_set'] = pd.to_datetime(df_blanket['1_hour_before_set'])
C:\Users\asobchen\AppData\Local\Temp\ipykernel_21444\619061979.py:86: SettingWithCopyWarning: 
A value is trying to 

'---------'